# KERI Basics

## Here we explore what Cryptographic Root of Trust and how it relates to:
* Private Keys
* Public Keys
* Identifiers

### Pysodium
Pysodium is a library that we'll use to produce the cryptographic primitives that we need. Lets install that first:

In [1]:
%pip install -q pysodium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import string
import secrets
import pysodium

#salt is bytes 16 byte long root cryptomatter from which seeds for Signers in list are derived
salt = pysodium.randombytes(pysodium.crypto_pwhash_SALTBYTES)
print("The random/entropy as bytes: ",salt)
assert len(salt) == 16

The random/entropy as bytes:  b'X\xad\x82\xb95\xf1\xabN\xafM\xb5\x9f\x9c*\xa9S'


In [3]:
seed = pysodium.crypto_pwhash(outlen=32,
                                      passwd="0",
                                      salt=salt,
                                      opslimit=2,  # pysodium.crypto_pwhash_OPSLIMIT_INTERACTIVE,
                                      memlimit=67108864,  # pysodium.crypto_pwhash_MEMLIMIT_INTERACTIVE,
                                      alg=pysodium.crypto_pwhash_ALG_ARGON2ID13)
print("Stretched the entropy with Argon2id",seed)

Stretched the entropy with Argon2id b'\x9428\xba\x93E\x8e\xe2=C&\xd6\x89\xf4\x9aYe\xf2]\xa8\xef\x91\x84Zxn\xa0N\x01\x02e\xbb'


In [4]:
%pip install -q keri

Note: you may need to restart the kernel to use updated packages.


In [5]:
# see https://github.com/WebOfTrust/keripy/blob/development/src/keri/core/coring.py
from keri.core import (coring)
#non-transferrable, basic derivation (simplest)
signer = coring.Signer(raw=seed, transferable=False)
# fetch sigkey as private key
print("Private (signing) key of signer, encoded as base64",signer.qb64)
# fetch verkey as public key
print("Public (verification) key of signer, encoded as base64",signer.verfer.qb64)
assert signer.verfer.code == coring.MtrDex.Ed25519N

Private (signing) key of signer, encoded as base64 AJQyOLqTRY7iPUMm1on0mlll8l2o75GEWnhuoE4BAmW7
Public (verification) key of signer, encoded as base64 BK9l9vbX7b2quhsf8azwUj9AMbp9BL4H-rY5izRGx5lb


In [6]:
message = "Sign me, please"
#In this case a Cigar in KERI
#see libsodium/pysodium crypto_sign_seed_keypair
signature = signer.sign(message)
print("Signature is",signature.qb64b)

Signature is b'0BB5dUrokhX1kwVZ_bYM94Ip1G3otmaukc9qvudpmYUrtcBLIGHjRtsZaS_oA3qH3qjwls-u371_sFRHBAvBrgwP'


In [7]:
result = signer.verfer.verify(signature.raw,message)
print("Verified message?",result)
assert result

Verified message? True


In [8]:
from keri.core.coring import Verfer, Prefixer, MtrDex 
verfer = signer.verfer
verkey = verfer.raw
verfer = Verfer(raw=verkey, code=MtrDex.Ed25519N)
ked = dict(k=[verfer.qb64], n="", t="icp", i=verfer.qb64)
prefixer = Prefixer(ked=ked, code=MtrDex.Ed25519N)  # verfer code match code and pre code
assert prefixer.qb64 == verfer.qb64
assert prefixer.verify(ked=ked) == True
assert prefixer.verify(ked=ked, prefixed=True) == True

In [9]:
import keri.core.eventing as eventing
# code marks this identifier as basic
#srdr = eventing.incept(keys=[signer.verfer.qb64], code=coring.MtrDex.Ed25519)  
#print(srdr.raw.decode("utf-8"))